# introdução ao Scikit-Learn e avaliação do modelo

O que veremos neste caderno:

* Explicar a variável de resposta;

* Descrever as implicações de dados desbalanceados na classificação binária;

* Dividir os dados em conjuntos de treinamento e teste;

* Descrever o ajuste do modelo no scikit-learn;

* Derivar várias métricas para a classificação binária;

* Criar uma curva ROCe uma curva precision-recall.

Este caderno concluirá a análise exploratória incial e apresentará novas ferramentas para execução da avaliação do modelo.

Há vários critérios importantes para a avaliação de modelos que são considerados conhecimento padrão em ciência de dados e machine learning. Abordaremos aqui algumas das mais usadas métricas de desempenho de modelos de classificação, para construirmos uma base sólida.

## Examinando a variável de resposta e concluindo a exploração inicial

Nosso objetivo é criar um modelo preditivo que a variável de resposta seja uma flag SIM/NÃO. Esse problema se chama tarefa de **classificação binária**.

Em nossos dados rotulados, consideramos que as amostras (contas) que ficarão inadimplentes (isto é, com status **default payment next month** | falta de pagamento no próximo mês = 1) pertencem à **classe positiva**, enquanto as que ficarão pertencem à **classe negativa**.

A informação mais importante que devemos examinar com relação à resposta de um problema de classificação binária é: **QUAL É A PROPORÇÃO DA CLASSE POSITIVA?**



Antes de fazer essa verificação, precisamos carregarmos os pacotes requeridos neste caderno

In [1]:
import numpy as np                 # pacote para cálculos númericos
import pandas as pd                # pacote preparação dos dados
import matplotlib.pyplot as plt    # pacote de plotagem

# definições de renderização de plotagens
%matplotlib inline
import matplotlib as mpl           # adiciona a funcionalidade de plotagem
mpl.rcParams['figure.dpi'] = 400   # figuras em alta resolução

Agora, precisamos importar a versão limpa dos dados trabalhados no caderno anterior

In [2]:
df = pd.read_csv('C:/Users/Renato/Dropbox/pos_usp/usp_data_open/data_science/projetos_de_ciencias_de_dados_com_python/Data/Chapter_1_cleaned_data.csv')

In [3]:
# Verificando dados da importação do dataset

df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26664 entries, 0 to 26663
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          26664 non-null  object
 1   LIMIT_BAL                   26664 non-null  int64 
 2   SEX                         26664 non-null  int64 
 3   EDUCATION                   26664 non-null  int64 
 4   MARRIAGE                    26664 non-null  int64 
 5   AGE                         26664 non-null  int64 
 6   PAY_1                       26664 non-null  int64 
 7   PAY_2                       26664 non-null  int64 
 8   PAY_3                       26664 non-null  int64 
 9   PAY_4                       26664 non-null  int64 
 10  PAY_5                       26664 non-null  int64 
 11  PAY_6                       26664 non-null  int64 
 12  BILL_AMT1                   26664 non-null  int64 
 13  BILL_AMT2                   26664 non-null  in

Para encontrar a proporção da classe postiva, precisamos apenas obter a média da variável de resposta no dataset inteiro

In [40]:
df['default payment next month'].mean()

0.2217971797179718

Acima saída da taxa de inadiplência.

Também é útil verificar o número de amostras em cada classe.

In [41]:
# Balanceamento de classes da variável de resposta

df.groupby('default payment next month')['ID'].count()

default payment next month
0    20750
1     5914
Name: ID, dtype: int64

A média das contas que ficaram inadimplentes é de 22 %, o que representa 5914 contas do portfólio (de acordo com a operação **[groupby/count](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.count.html)**).

A proporção de amostras na classe positiva (inaimplência = 1), também chamada de **fração da classe**, é uma estatística importante.

Na classificação binária, os datasets são descritos considerando-se se eles são **balanceados** ou **desbalanceados**.

A maioria dos modelos de classificação de machine learning é projetada para operar com dados balanceados: uma divisão 50/50 entre as classes.

Na prática, raramente essa divisão (50/50) é alcançada. Assim, há vários métodos que têm como objetivo lidar com dados desbalanceados. São eles:

- **Subamostragem (undersampling)** da classe majoritária: eliminar aleatoriamente amostras da classe majoritária até as frações serem iguais (ou menos desbalanceadas);
- **Sobreamonstragem (oversampling)** da classe minoritária: adicionar aleatoiamente amostras duplicadas da classe minoritáia para atingir o mesmo objetivo;
- **Ponderação de amostragem (weighting samples)**: esse método é executado como parte da etapa de treinamento, para que a classe minoritária receba coletivamente a mesma 'ênfase' da classe majoritária no modelo ajustado. O efeito é o mesmo do oversampling).



# Introdução ao scikit-learn

Scikit-learn é um dos pricipais pacotes de machine learning para Python (com exceção dos de deep learning).

Nesta seção, será ilustrado a sintaxe usando um modelo de **regressão logística**. Regressão logística é um dos mais simples modelos de classificação (examinaremos os detalhes matemáticos e seu funcionamento mais adiante).



Anteriormente, nos familiarizamos com o conceito de treinar um algoritmo a partir de dados para usar o modelo treinado a fim de fazer previsões com novos dados.

O scikit-learn encapsulo essas funconalidades b´saicas no método **.fit** para treinamento de modelos e no método **.predict** para a execução de previsões.

Devido à sintaxe consistente, você pode chamar **.fit** e **.predict** em qualquer modelo do scikit-learn, seja de regressão linear ou de árvores de classificação.



A primeira etapa é selecionar algum modelo, nesse exemplo uma **regressão logística**, e instanciá-lo a partir da **classe** fornecida pelo scikit-learn disponibilizada e criando um **objeto** útil a partir dele.

Podemos treinar esse objetivo com seus dados e salvá-lo em disco para uso posterior.

Primeiro temos que importar a classe:

In [42]:
#Import the class

from sklearn.linear_model import LogisticRegression

O código que instancia a **classe** para criar um objeto é:

In [43]:
#Create an object

my_lr = LogisticRegression()

In [44]:
#Examine it

my_lr

LogisticRegression()

Na saída acima deveria conter uma extensa saída (a saída deveria ser a mesma da linha abaixo). 

In [45]:
# Instancie ao especificar argumentos de palavra-chave

my_new_lr = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

#5/2020: updated multi_class='warn' to multi_class='auto'

Ainda que o objeto que criamos aqui em **my_new_lr** seja idêntico a **my_lr**, agir dessa maneira explícita será particularmente útil quando estivermos começando a travar contato com diferentes tipos de modelos.

Quando estiver mais familizarido, poderemos executar a instanciação com as opções padrão e fazer alterções posteriormente conforme necessário.

*Há uma explicação na página 73 do livro de como alterar este padrão.*

In [49]:
my_new_lr.C = 0.1
my_new_lr.solver = 'liblinear'

_______________

Como sabemos, os algoritmos de aprendizado supervisionado dependem de dados rotulados. 

Isso significa que pecisamos tanto das características, normalmente contidas em uma variável **X**, quando das respostas correspondentes, em uma variável chamada **y**.

Tomaremos emprestadas de nosso dataset as 10 primeiras linhas amostras de uma única característica, e a resposta, para ilustrar

In [50]:
X = df['EDUCATION'][0:10].values.reshape(-1,1)

X

array([[2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [3],
       [3]], dtype=int64)

Os 10 primeiros valores correspondentes da variável de resposta podemser obtidos assim:


In [51]:
y = df['default payment next month'][0:10].values

y

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

O scikit-learn espera que a primeira dimensão (isto é, o número de linhas) do array de característcas seja o número de amostras, logo, tivemos de fazer esse redimensionamento para X, mas não para y.

<img src="introducao_data_science_python/imgs/img_array.png">

Agora usaremos esses dados para ajustar nossa regressão logística. Isso pode ser feito com uma única linha:

In [52]:
# ajustando um modelo no scikit-learn

my_new_lr.fit(X, y)

LogisticRegression(C=0.1, solver='liblinear')

Agora o objetivo de modelo **my_new_lr** é um modelo treinado.

Podemos dizer que essa alteração ocorreu **in loco**, já que não foi criado um novo objeto. 

Já podemos usar nosso modelo reinado para fazer previsões para novas características que o modelo ainda não 'viu'. Tentaremos usar as p´roximas 10 linhas da característica EDUCATION.

In [53]:
new_X = df['EDUCATION'][10:20].values.reshape(-1,1)

new_X

array([[3],
       [1],
       [2],
       [2],
       [1],
       [3],
       [1],
       [1],
       [1],
       [3]], dtype=int64)

As previsções são feitas assim:

In [54]:
# previsões para as novas características

my_new_lr.predict(new_X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

Também podemos visualizar quais são os valores reais correspondentes a essas previsões:


In [55]:
df['default payment next month'][10:20].values

array([0, 0, 0, 1, 0, 0, 1, 0, 0, 0], dtype=int64)